In [1]:
from sklearn.metrics import mean_squared_error, roc_auc_score
import numpy as np
import torch
from trees import construct_tree, TreeDataset, do_fl_partitioning

In [2]:
from sklearn.model_selection import StratifiedKFold
random_state = np.random.randint(1000)
def fl_split(split, x_train, y_train):

    x_eq = []
    y_eq = []

    n_splits = 50
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    skf.get_n_splits(x_train, y_train)

    for i, (_, test_index) in enumerate(skf.split(x_train, y_train)):
        x_eq.append(x_train[test_index])
        y_eq.append(y_train[test_index])

    x_split = []
    y_split = []

    acc = 0
    for s in split:
        x_split.append(torch.cat(x_eq[acc:acc+int(s*n_splits)], 0))
        y_split.append(torch.cat(y_eq[acc:acc+int(s*n_splits)], 0))
        acc += int(s*n_splits)

    return x_split, y_split

In [3]:
# Local models
X = torch.load('./dataset3/X.pt')
y = torch.load('./dataset3/y.pt')
X_tensor = torch.tensor(X)
y_tensor = torch.reshape(torch.tensor(y), (-1, 1))
print(y_tensor.shape)

torch.Size([150000, 1])


C:\Users\Shuyao\AppData\Local\Temp\ipykernel_33712\2612116312.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_tensor = torch.tensor(X)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_33712\2612116312.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_tensor = torch.reshape(torch.tensor(y), (-1, 1))


In [4]:
from imblearn.over_sampling import RandomOverSampler


def run_local(split, X=X, y=y):
    mses = []
    aucs = []
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    for fold_idx, (train_idx, test_idx) in enumerate(kf.split(X, y)):
        print(f"Fold {fold_idx + 1}:")

        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        X_train = torch.tensor(X_train)
        X_test = torch.tensor(X_test)
        y_train = torch.reshape(torch.tensor(y_train), (-1, 1))
        y_test = torch.reshape(torch.tensor(y_test), (-1, 1))


        x_split, y_split = fl_split(split, X_train, y_train)

        for X_train, y_train in zip(x_split, y_split):

            ros = RandomOverSampler(random_state=random_state)
            X_train, y_train = ros.fit_resample(X_train, y_train)
            X_train = torch.from_numpy(X_train)

            y_train = torch.reshape(torch.from_numpy(y_train), (-1, 1))
            y_test = torch.reshape(y_test, (-1, 1))

            global_tree = construct_tree(X_train, y_train, learning_rate=0.1, max_depth=5, n_estimators=80,
               min_child_weight=3, subsample=0.8,colsample_bytree=1, gamma=0)

            print(X_test.shape)

            preds_test = global_tree.predict_proba(X_test)
            pred = preds_test[:, 1]
            auc = roc_auc_score(y_test, pred)
            mse = mean_squared_error(y_test, pred)

            aucs.append(auc)
            mses.append(mse)
    print('split',split)
    print('mse',mses)
    print('auc',aucs)

In [5]:
split = [1]
run_local(split)

Fold 1:


C:\Users\Shuyao\AppData\Local\Temp\ipykernel_33712\1826658617.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_33712\1826658617.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_33712\1826658617.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.reshape(torch.tensor(y_train), (-1, 1))
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_33712\1826658617.py:22: UserWarning:

torch.Size([30000, 10])
Fold 2:


C:\Users\Shuyao\AppData\Local\Temp\ipykernel_33712\1826658617.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_33712\1826658617.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_33712\1826658617.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.reshape(torch.tensor(y_train), (-1, 1))
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_33712\1826658617.py:22: UserWarning:

torch.Size([30000, 10])
Fold 3:


...

torch.Size([30000, 10])
Fold 4:


C:\Users\Shuyao\AppData\Local\Temp\ipykernel_33712\1826658617.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_33712\1826658617.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_33712\1826658617.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.reshape(torch.tensor(y_train), (-1, 1))
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_33712\1826658617.py:22: UserWarning:

torch.Size([30000, 10])
Fold 5:


C:\Users\Shuyao\AppData\Local\Temp\ipykernel_33712\1826658617.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_33712\1826658617.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_33712\1826658617.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.reshape(torch.tensor(y_train), (-1, 1))
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_33712\1826658617.py:22: UserWarning:

torch.Size([30000, 10])
split [1]
mse [0.13814794, 0.14080842, 0.13772589, 0.13901256, 0.14117733]
auc [0.8644127095743561, 0.864220445492805, 0.8706514121910797, 0.8593546140008079, 0.8675686564977091]


In [15]:
split = [0.5, 0.5]
run_local(split)

Fold 1:


C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.reshape(torch.tensor(y_train), (-1, 1))
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:23: UserWarning:

Fold 2:


C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.reshape(torch.tensor(y_train), (-1, 1))
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:23: UserWarning:

Fold 3:


...

Fold 4:


C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.reshape(torch.tensor(y_train), (-1, 1))
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:23: UserWarning:

Fold 5:


C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.reshape(torch.tensor(y_train), (-1, 1))
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:23: UserWarning:

split [0.5, 0.5]
mse [0.13554646, 0.13511342, 0.13707313, 0.13913219, 0.13543665, 0.13344623, 0.13649379, 0.1351098, 0.1379548, 0.13914117]
auc [0.8596437325873348, 0.8592651708374199, 0.861936576668705, 0.8617064857057214, 0.8682442046339767, 0.8683663675246605, 0.8571170573298847, 0.8550013784969617, 0.8650506347098854, 0.8652671500388162]


In [16]:
split = [0.6, 0.4]
run_local(split)

Fold 1:


C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.reshape(torch.tensor(y_train), (-1, 1))
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:23: UserWarning:

Fold 2:


C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.reshape(torch.tensor(y_train), (-1, 1))
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:23: UserWarning:

Fold 3:


...

Fold 4:


C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.reshape(torch.tensor(y_train), (-1, 1))
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:23: UserWarning:

Fold 5:


C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.reshape(torch.tensor(y_train), (-1, 1))
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:23: UserWarning:

split [0.6, 0.4]
mse [0.1362956, 0.13350506, 0.1392099, 0.13676855, 0.13545832, 0.13352124, 0.13739894, 0.13408263, 0.13946009, 0.13640673]
auc [0.8607128799356022, 0.8582053546440767, 0.8635925367862716, 0.8612915291695747, 0.8691700023026911, 0.8683821790406998, 0.8568885074329715, 0.8535055734480552, 0.8654039664190123, 0.8613872890554467]


In [17]:
split = [0.8, 0.2]
run_local(split)

Fold 1:


C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.reshape(torch.tensor(y_train), (-1, 1))
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:23: UserWarning:

Fold 2:


C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.reshape(torch.tensor(y_train), (-1, 1))
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:23: UserWarning:

Fold 3:


...

Fold 4:


C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.reshape(torch.tensor(y_train), (-1, 1))
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:23: UserWarning:

Fold 5:


C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.reshape(torch.tensor(y_train), (-1, 1))
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:23: UserWarning:

split [0.8, 0.2]
mse [0.1383434, 0.123614565, 0.14051673, 0.12829821, 0.13706769, 0.12562163, 0.13847578, 0.12800004, 0.14078689, 0.13084194]
auc [0.8624255831491023, 0.8522075411259968, 0.8636973025553636, 0.8568017445224232, 0.8702995146532668, 0.8610974528315753, 0.857843763514949, 0.8483456299419339, 0.8664693419157945, 0.8580124345325293]


In [18]:
split = [0.34, 0.33, 0.33]
run_local(split)

split = [0.6, 0.2, 0.2]
run_local(split)

split = [0.8, 0.1, 0.1]
run_local(split)

Fold 1:


C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.reshape(torch.tensor(y_train), (-1, 1))
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:23: UserWarning:

Fold 2:


C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.reshape(torch.tensor(y_train), (-1, 1))
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:23: UserWarning:

Fold 3:


...

Fold 4:


C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.reshape(torch.tensor(y_train), (-1, 1))
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:23: UserWarning:

Fold 5:


C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.reshape(torch.tensor(y_train), (-1, 1))
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:23: UserWarning:

split [0.8, 0.1, 0.1]
mse [0.1383434, 0.11920269, 0.11671416, 0.14051673, 0.117068715, 0.11799401, 0.13706769, 0.11487291, 0.118996225, 0.13847578, 0.11802307, 0.118222006, 0.14078689, 0.11995363, 0.12274225]
auc [0.8624255831491023, 0.8417254844026897, 0.8382552296671464, 0.8636973025553636, 0.8463194220200525, 0.8517110420947096, 0.8702995146532668, 0.8546852817946204, 0.8540449732963536, 0.857843763514949, 0.8380507563026707, 0.8433698126535776, 0.8664693419157945, 0.8482339516452662, 0.8475992551217064]


In [19]:
split = [0.2, 0.2, 0.2, 0.2, 0.2]
run_local(split)

split = [0.6, 0.1, 0.1, 0.1, 0.1]
run_local(split)

split = [0.8, 0.05, 0.05, 0.05, 0.05]
run_local(split)

Fold 1:


C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.reshape(torch.tensor(y_train), (-1, 1))
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:23: UserWarning:

Fold 2:


C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.reshape(torch.tensor(y_train), (-1, 1))
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:23: UserWarning:

Fold 3:


...

Fold 4:


C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.reshape(torch.tensor(y_train), (-1, 1))
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:23: UserWarning:

Fold 5:


C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.reshape(torch.tensor(y_train), (-1, 1))
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:23: UserWarning:

split [0.8, 0.05, 0.05, 0.05, 0.05]
mse [0.1383434, 0.09970835, 0.10221057, 0.100774094, 0.09895958, 0.14051673, 0.10190161, 0.1008584, 0.09798835, 0.100531645, 0.13706769, 0.09742572, 0.104023255, 0.094115585, 0.101980075, 0.13847578, 0.09980843, 0.09763886, 0.10400569, 0.09919609, 0.14078689, 0.10760928, 0.09483427, 0.10495425, 0.101850815]
auc [0.8624255831491023, 0.8285668642425941, 0.8287916845306049, 0.8186949920403823, 0.8207497248199676, 0.8636973025553636, 0.8335141072127682, 0.8326409373957497, 0.8376590315602315, 0.8403735080943828, 0.8702995146532668, 0.8371505866517847, 0.8394286386195612, 0.8295511355563583, 0.834412014257979, 0.857843763514949, 0.8268755776214047, 0.8186614905850216, 0.8209417695268882, 0.8138772376078913, 0.8664693419157945, 0.8280293016342871, 0.8318898770220368, 0.835291998615713, 0.8181453937936014]


In [20]:
split = [0.6, 0.1, 0.1, 0.1, 0.1]
run_local(split)

Fold 1:


C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.reshape(torch.tensor(y_train), (-1, 1))
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:23: UserWarning:

Fold 2:


C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.reshape(torch.tensor(y_train), (-1, 1))
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:23: UserWarning:

Fold 3:


...

Fold 4:


C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.reshape(torch.tensor(y_train), (-1, 1))
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:23: UserWarning:

Fold 5:


C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.reshape(torch.tensor(y_train), (-1, 1))
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:23: UserWarning:

split [0.6, 0.1, 0.1, 0.1, 0.1]
mse [0.1362956, 0.12340555, 0.119984604, 0.11920269, 0.11671416, 0.1392099, 0.12001645, 0.12573688, 0.117068715, 0.11799401, 0.13545832, 0.11516599, 0.11976686, 0.11487291, 0.118996225, 0.13739894, 0.1182458, 0.120992206, 0.11802307, 0.118222006, 0.13946009, 0.12070212, 0.121659584, 0.11995363, 0.12274225]
auc [0.8607128799356022, 0.8414456833115713, 0.8412881043231668, 0.8417254844026897, 0.8382552296671464, 0.8635925367862716, 0.8486133567670393, 0.8456700363753948, 0.8463194220200525, 0.8517110420947096, 0.8691700023026911, 0.8524136346043268, 0.8544302309060354, 0.8546852817946204, 0.8540449732963536, 0.8568885074329715, 0.8398171297956217, 0.8406799219133805, 0.8380507563026707, 0.8433698126535776, 0.8654039664190123, 0.8426288449264409, 0.8449925195940315, 0.8482339516452662, 0.8475992551217064]


In [21]:
split = [0.1, 0.1, 0.1, 0.1, 0.1,
        0.1, 0.1, 0.1, 0.1, 0.1]
run_local(split)

Fold 1:


C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.reshape(torch.tensor(y_train), (-1, 1))
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:23: UserWarning:

Fold 2:


C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.reshape(torch.tensor(y_train), (-1, 1))
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:23: UserWarning:

Fold 3:


...

Fold 4:


C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.reshape(torch.tensor(y_train), (-1, 1))
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:23: UserWarning:

Fold 5:


C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.reshape(torch.tensor(y_train), (-1, 1))
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:23: UserWarning:

split [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
mse [0.11716657, 0.11849374, 0.12171871, 0.11591146, 0.117737584, 0.11939775, 0.12340555, 0.119984604, 0.11920269, 0.11671416, 0.118295915, 0.1186676, 0.12932354, 0.11991461, 0.1163183, 0.1244088, 0.12001645, 0.12573688, 0.117068715, 0.11799401, 0.115395054, 0.11810924, 0.11838953, 0.11965571, 0.12159849, 0.11144612, 0.11516599, 0.11976686, 0.11487291, 0.118996225, 0.11970679, 0.11644799, 0.12187697, 0.12092047, 0.11984751, 0.11761282, 0.1182458, 0.120992206, 0.11802307, 0.118222006, 0.121661976, 0.12049867, 0.117855504, 0.12015608, 0.12189518, 0.12475236, 0.12070212, 0.121659584, 0.11995363, 0.12274225]
auc [0.8452433654241962, 0.8401270166068202, 0.8375615455555174, 0.8470091618407619, 0.8411190038514876, 0.838427054693603, 0.8414456833115713, 0.8412881043231668, 0.8417254844026897, 0.8382552296671464, 0.845202772671821, 0.8448608341621389, 0.846360362711724, 0.8420971415005976, 0.8486227902292848, 0.8473886190043733, 0.8486133

In [22]:
split = [0.6, 0.05, 0.05, 0.05, 0.05, 0.04, 0.04, 0.04, 0.04, 0.04]
run_local(split)

Fold 1:


C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.reshape(torch.tensor(y_train), (-1, 1))
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:23: UserWarning:

Fold 2:


C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.reshape(torch.tensor(y_train), (-1, 1))
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:23: UserWarning:

Fold 3:


...

Fold 4:


C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.reshape(torch.tensor(y_train), (-1, 1))
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:23: UserWarning:

Fold 5:


C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.reshape(torch.tensor(y_train), (-1, 1))
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:23: UserWarning:

split [0.6, 0.05, 0.05, 0.05, 0.05, 0.04, 0.04, 0.04, 0.04, 0.04]
mse [0.1362956, 0.1068783, 0.10500127, 0.10529941, 0.10581757, 0.0974041, 0.09970835, 0.10221057, 0.100774094, 0.09895958, 0.1392099, 0.10069682, 0.10447811, 0.103195444, 0.11192394, 0.106811956, 0.10190161, 0.1008584, 0.09798835, 0.100531645, 0.13545832, 0.10005179, 0.09445961, 0.096184894, 0.10498802, 0.10624178, 0.09742572, 0.104023255, 0.094115585, 0.101980075, 0.13739894, 0.09980704, 0.098227054, 0.10387419, 0.10287014, 0.106215395, 0.09980843, 0.09763886, 0.10400569, 0.09919609, 0.13946009, 0.10271255, 0.10328829, 0.10438525, 0.1106437, 0.103606135, 0.10760928, 0.09483427, 0.10495425, 0.101850815]
auc [0.8607128799356022, 0.8256285298565973, 0.82859201740353, 0.8338038057720815, 0.8283014783612298, 0.8259942434609439, 0.8285668642425941, 0.8287916845306049, 0.8186949920403823, 0.8207497248199676, 0.8635925367862716, 0.8269992370386767, 0.8299613976311231, 0.8364861110306926, 0.8221661420658035, 0.8311283498701006, 

In [23]:
split = [0.8, 0.03, 0.03, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02]
run_local(split)

Fold 1:


C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.reshape(torch.tensor(y_train), (-1, 1))
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:23: UserWarning:

Fold 2:


C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.reshape(torch.tensor(y_train), (-1, 1))
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:23: UserWarning:

Fold 3:


...

Fold 4:


C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.reshape(torch.tensor(y_train), (-1, 1))
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:23: UserWarning:

Fold 5:


C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test)
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.reshape(torch.tensor(y_train), (-1, 1))
C:\Users\Shuyao\AppData\Local\Temp\ipykernel_26868\1736011833.py:23: UserWarning:

split [0.8, 0.03, 0.03, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02]
mse [0.1383434, 0.08589134, 0.08536178, 0.08539521, 0.089931004, 0.089007616, 0.08644354, 0.08536266, 0.084270865, 0.08526722, 0.14051673, 0.09406823, 0.085136175, 0.089886636, 0.086324245, 0.080077, 0.08948174, 0.08275824, 0.08992946, 0.09379348, 0.13706769, 0.086065225, 0.081671745, 0.08967126, 0.0946081, 0.08600734, 0.08188916, 0.08497879, 0.08709027, 0.08548024, 0.13847578, 0.085814185, 0.088723436, 0.09033298, 0.07927249, 0.09086723, 0.088725306, 0.08496987, 0.091522634, 0.0863883, 0.14078689, 0.09151107, 0.09282598, 0.08686911, 0.08241353, 0.0926735, 0.08988996, 0.084276386, 0.0878917, 0.089749075]
auc [0.8624255831491023, 0.8209568265981508, 0.8166163259168695, 0.8267283952244147, 0.8234653402085663, 0.8060150832777084, 0.7942951081740847, 0.8016523320657447, 0.8125519846832296, 0.8194289835359252, 0.8636973025553636, 0.8121536665569511, 0.8319318599375825, 0.8234972222952174, 0.8012486376486004, 0.820171574991793